In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [1]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(2)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

## Очистка пропущенных значений

Пропущенные значения содержаться в признаках lat и lng.
Посмотрим для каких адресов отелей эти значения не заполнены.

In [ ]:
mask1 = df['lat'].isnull()
mask2 = df['lng'].isnull()
df[mask1 & mask2]['hotel_address'].value_counts()

In [ ]:
#выделим признак страны из адреса
df['country'] = df['hotel_address'].apply(
    lambda x: x.split()[-1] if x.split()[-1] != 'Kingdom'
    else 'UK')
df['country'].value_counts()

In [ ]:
#выделим признак города из адреса отеля
df['city'] = df['hotel_address'].apply(
    lambda x: x.split()[-2] if x.split()[-1] != 'Kingdom'
    else x.split()[-5])
df['city'].value_counts()

In [ ]:
df.info()

In [ ]:
#расчитаем среднее значение 'lat','lng' для каждого города
median_country = df.groupby(by='city').median().loc[:,['lat','lng']]
median_country

In [ ]:
#заполним пропуски средними значениями по городу
df_new = df.merge(median_country, left_on='city', right_on='city')
df['lat'] = df['lat'].fillna(df_new['lat_y'])
df['lng'] = df['lng'].fillna(df_new['lng_y'])
df.info()

Получили датасет без пропущенных значений.

## Генерация и преобразование признаков

Выше мы уже выделили из адреса признаки страны и города. Так эти признаки имеют высокую корреляцию, в дальнейшем будем преобразовывать только признак города.

In [ ]:
#преобразуем город в OneHot признак
df = pd.get_dummies(df, columns=['city'])
df.columns

Преобразуем признак 'tags', выделив в отдельные признаки наиболее часто встречающиеся теги.

In [ ]:
#преобразуем признак tags в список тэгов 
df['tags'] = df['tags'].apply(
    lambda x: x.replace(
        '[',''
        ).replace(
            ']',''
            ).replace(
                "' ",''
                ).replace(
                    " '",''
                    ).split(', ')
)

df['tags'].head()

In [ ]:
#посчитаем количество уникальных тэгов
len(df['tags'].explode().unique())

In [ ]:
#выберем топ тэгов с количеством
df['tags'].explode().value_counts().head(10)

In [ ]:
#сохраним наиболее часто встречающиеся тэги
top_of_tags = df['tags'].explode().value_counts().head(10).index
top_of_tags

In [ ]:
#создадим признаки для наиболее часто встречающихся тэгов
for tag in top_of_tags:
    df['tag_'+ tag] = df['tags'].apply(
        lambda x: 1 if tag in x else 0
    )
df.info()

Создадим новые признаки из признака 'negative_review'

In [ ]:
#преобразуем признак negative_review в список слов 
df['negative_review'] = df['negative_review'].apply(
    lambda x: x.lower().split()
)
df['negative_review'].head()

In [ ]:
#посмотрим наиболее часто встречающиеся слова, исключив из них слова 
#длиной 3 и менее символов, так как это в основном артикли, союзы, местоимения
df_negative = pd.DataFrame(
    df[df['review_total_negative_word_counts'] > 0]['negative_review'].explode()
)
df_negative['len'] = df_negative['negative_review'].apply(lambda x: len(str(x)))
df_negative[df_negative['len'] > 3]['negative_review'].value_counts().head(20)

In [ ]:
#заполним вручную список часто встречающихся слов в негативном отзыве
top_negative_words = [
    'room', 'very', 'hotel', 'breakfast', 'small', 'staff', 'nothing', 'rooms', 'only','little'
    ]

In [ ]:
#создадим признаки для наиболее часто встречающихся слов в negative_review
for negative_review in top_negative_words:
    df['negative_'+ negative_review] = df['negative_review'].apply(
        lambda x: 1 if negative_review in x else 0
    )
df.info()

 Создадим новые признаки из признака 'positive_reviw'

In [ ]:
#преобразуем признак positive_review в список слов 
df['positive_review'] = df['positive_review'].apply(
    lambda x: x.lower().split()
)
df['positive_review'].head()

In [ ]:
#смотрим часто встречающиеся слова
df_positive = pd.DataFrame(
    df[df['review_total_positive_word_counts'] > 0]['positive_review'].explode()
)
df_positive['len'] = df_positive['positive_review'].apply(lambda x: len(str(x)))
df_positive[df_positive['len'] > 3]['positive_review'].value_counts().head(20)

In [ ]:
top_positive_words = ['staff', 'location', 'room', 'hotel' , 'breakfast', 'rooms' 
    'nice', 'clean', 'very', 'excellent', 'comfortable', 'good', 
    'great', 'friendly', 'lovely','helpful'
    ]

In [ ]:
#создадим признаки для наиболее часто встречающихся слов в positive_review
for positive_review in top_positive_words:
    df['positive_'+ positive_review] = df['positive_review'].apply(
        lambda x: 1 if positive_review in x else 0
    )
df.info()

Преобразуем признак 'reviewer_nationality'

In [ ]:
#смотрим часто встречающиеся национальности
df['reviewer_nationality'].value_counts().head(10)

In [ ]:
#выделим топ 4
top_nationality = df['reviewer_nationality'].value_counts().head(4).index
top_nationality

In [ ]:
df['reviewer_nationality'] = df['reviewer_nationality'].apply(
    lambda x: x if x in top_nationality else 'Other'
)
df['reviewer_nationality'].value_counts()

In [ ]:
fig = px.box(df, x='reviewer_score', y='reviewer_nationality')
fig.show()

На графике распределения оценок в разрезе национальности видно, что средняя оценка отличается, то есть этот признак вероятно влияет на выставляемую оценку.

In [ ]:
#закодируем признак национальности в OneHot
df = pd.get_dummies(df, columns=['reviewer_nationality'])
df.info()

Преобразуем принаки даты: 'days_since_review' и 'review_date'

In [ ]:
#преобразуем признак 'days_since_review' в числовой
df['days_since_review'] = df['days_since_review'].apply(lambda x: int(x.split()[0]))
df['days_since_review'].head()

In [ ]:
df['review_date'] = pd.to_datetime(df['review_date'])

In [ ]:
#выделим признаки из даты
df['month'] = df['review_date'].dt.month
df['year'] = df['review_date'].dt.year

In [ ]:
fig = px.box(df, x='reviewer_score', y='month', orientation='h')
fig.show()

Визуально, средняя оценка выше при оценке в первые три месяца года, и не отличается для остальных месяцев, то есть в первый квартал года. Выделим признак квартала.

In [ ]:
df['quarter'] = df['review_date'].dt.quarter

In [ ]:
fig = px.box(df, x='reviewer_score', y='quarter', orientation='h')
fig.show()

Для первого квартала видим отличие. 

In [ ]:
#закодируем признак квартала
df = pd.get_dummies(df, columns=['quarter'])

In [ ]:
fig = px.box(df, x='reviewer_score', y='year', orientation='h')
fig.show()

Графики распределения оценок в разрезе лет существенно не отличаются, можем предположить, что год выставления оценки не влияет на размер, т.е. этот признак не является значимым.

## Нормализация признаков

In [ ]:
#проведем стандартизацию/нормализацию для числовых признаков
num_cols = ['additional_number_of_scoring', 'average_score',
       'review_total_negative_word_counts', 'total_number_of_reviews',
       'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'lat',
       'lng','days_since_review']

In [ ]:
import scipy.stats as stats

# задаём уровень значимости
alpha = 0.05 

# функция для принятия решения о нормальности
def decision_normality(p):
    print('p-value = {:.3f}'.format(p))
    if p <= alpha:
        print('p-значение меньше, чем заданный уровень значимости {:.2f}. Распределение отлично от нормального'.format(alpha))
    else:
        print('p-значение больше, чем заданный уровень значимости {:.2f}. Распределение является нормальным'.format(alpha))

In [ ]:
#проверим распределение числовых признаков на нормальность с помощью теста Шапиро — Уилка

for col in num_cols:
    print('Признак ' + col)
    result = stats.shapiro(df[col])
    decision_normality(result[1])

Так как все признаки распределены не нормально, используем нормализацию для преобразования. Будем использовать нормализатор RobustScaler, так как мы не выполняли очистку данных

In [ ]:
from sklearn import preprocessing

# инициализируем нормализатор RobustScaler
r_scaler = preprocessing.RobustScaler()
# кодируем исходный датасет
df[num_cols] = r_scaler.fit_transform(df[num_cols])

In [ ]:
df[num_cols].head()

## Отбор признаков

In [ ]:
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in df.columns if df[s].dtypes == 'object']
df.drop(object_columns, axis = 1, inplace=True)
df.drop('review_date', axis = 1, inplace=True)

In [ ]:
df.info()

Проведем анализ мультиколлинеарности

In [ ]:
df_corr = df.drop(['sample'], axis=1).corr()
# График - тепловая матрица с коэффициентами корреляции не менее 0.7
fig = sns.heatmap(
    df_corr[df_corr.abs() >= 0.7],
    linewidth=0.1, 
    linecolor='gray', 
    annot = True
)

fig.set_title('Мультиколлинеарность признаков', fontsize=20);

Сформируем список признаков для удаления на основании анализа мультиколлинеарности:
- 'additional_number_of_scoring' - признак коррелирует с признаком 'total_number_of_review'. Оставляем признак 'total_number_of_review', так как он кажется более информативным
- признаки городов ('city_Amsterdam', 'city_Barcelona', 'city_London', 'city_Milan', 'city_Paris', 'city_Vienna') коррелируют с признаками 'lat', 'lng'. Оставляем признаки координат, так как они более точные и в случае расширения списка городов не приведут к увеличению количетва признаков
- 'tag_Business trip' - признак коррелирует с признаком 'tag_Leisure trip'
- 'reviewer_nationality_Other' - признак коррелирует с др признаками национальностей
- 'year' - признак коррелирует с признаком 'days_since_review' и так как выше мы не увидели зависимости оценки от года выставления, признак года можно удалить
- 'month' - коррелирует с признаками квартала. Попробуем оставить признак квартала


In [ ]:
high_corr = [
    'additional_number_of_scoring', 'city_Amsterdam', 'city_Barcelona', 
    'city_London', 'city_Milan', 'city_Paris', 'city_Vienna', 'tag_Business trip',
    'reviewer_nationality_Other', 'year', 'month'
    ]

In [ ]:
#удаляем выбранные признаки
df = df.drop(high_corr, axis=1)

In [ ]:
df.columns

In [ ]:
# Теперь выделим тестовую часть
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# непрерывные признаки
num_cols = ['average_score', 'review_total_negative_word_counts',
       'total_number_of_reviews', 'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'days_since_review',
       'lat', 'lng']

# категориальные признаки
cat_cols = ['tag_Leisure trip',
       'tag_Submitted from a mobile device', 'tag_Couple',
       'tag_Stayed 1 night', 'tag_Stayed 2 nights', 'tag_Solo traveler',
       'tag_Stayed 3 nights', 'tag_Group', 'tag_Family with young children',
       'negative_room', 'negative_very', 'negative_hotel',
       'negative_breakfast', 'negative_small', 'negative_staff',
       'negative_nothing', 'negative_rooms', 'negative_only',
       'negative_little', 'positive_staff', 'positive_location',
       'positive_room', 'positive_hotel', 'positive_breakfast',
       'positive_roomsnice', 'positive_clean', 'positive_very',
       'positive_excellent', 'positive_comfortable', 'positive_good',
       'positive_great', 'positive_friendly', 'positive_lovely',
       'positive_helpful', 'reviewer_nationality_ Australia ',
       'reviewer_nationality_ Ireland ',
       'reviewer_nationality_ United Kingdom ',
       'reviewer_nationality_ United States of America ', 'quarter_1',
       'quarter_2', 'quarter_3', 'quarter_4']

In [ ]:
y=y.astype('int')

from sklearn.feature_selection import chi2 # хи-квадрат

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
fig, ax = plt.subplots(figsize=(15, 20))
imp_cat.plot(kind = 'barh')

In [ ]:
from sklearn.feature_selection import f_classif # anova

imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

На основе анализа важности признаков удалим признаки кварталов, оставив только признак первого квартала, так как именно для него отличалось среднее значение оценки в визуальном анализе выше. 

In [ ]:
columns_drop = ['quarter_2', 'quarter_3', 'quarter_4']

In [ ]:
df = df.drop(columns_drop, axis = 1) 

In [ ]:
y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)
      
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# # Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений в отношении фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

В рамках работы над текущим проектом были выполнены следующие задачи:
 - Очистка данных от пропущенных значений и неинформативных признаков
 - Выделение из имеющихся данных новых признаков
 - Преобразование признаков в удобный формат для машинного обучения
 - Отбор наиболее значимых признаков
 - Построение и обучение модели

Результаты:
1. На тестовых данных полученный результат метрики MAPE: 0.12835925534440035
Он превышает результат полученный до EDA примерно на 2 процентных пункта.
2. Опытным путем установлено, что наибольшее повышение точности модели удалось получить именно за счет генерации новых признаков из имеющихся данных.
3. Отбор признаков на этапе анализа важности признаков для целевой переменной принес вклад не более 0.2 процентных пунктов. При этом удаление большего количества признаков даже ухудшает качество модели